# ロジスティック回帰(Logistic Regression)
ロジスティック回帰は、線形回帰を拡張し、シグモイド関数またはソフトマックス関数を用いて確率的な出力を生成するモデル。  
出力が 2 クラスならシグモイド関数、多クラス分類なら ソフトマックス関数 を適用。 
目的関数には、クロスエントロピー損失関数 (CrossEntropyLoss) を使用。
### 特徴
単純なデータに対して有効。  
学習が高速で、計算コストが低い。  
どの特徴量がどれくらい影響を与えているかが分かりやすい。  
正則化で不要な特徴量を抑制したり、過学習を防ぐことができる。
### メリット
計算が速い  
確率的な解釈が可能  
単純で解釈しやすい  
正則化で過学習を抑えられる  
### デメリット
線形分離しかできない  
多クラス分類には拡張が必要  
大規模データでは性能が低下

In [1]:
import torch
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # CrossEntropyLoss を使う場合はソフトマックスを明示的に入れない
        return self.linear(x)



In [3]:
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# データの用意
iris = load_iris()
X, y = iris.data, iris.target  # X.shape: (150, 4), y.shape: (150,)

# 訓練・テスト分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 特徴量の標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tensor に変換
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# モデル定義
input_dim = X_train.shape[1]  # 4
output_dim = len(torch.unique(y_train))  # 3
model = LogisticRegression(input_dim, output_dim)

# 損失関数と最適化手法
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 学習
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        # 評価
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            _, predicted = torch.max(test_outputs, 1)
            accuracy = (predicted == y_test).float().mean()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Test Acc: {accuracy:.4f}")


Epoch [10/50], Loss: 1.2636, Test Acc: 0.1333
Epoch [20/50], Loss: 1.1563, Test Acc: 0.2667
Epoch [30/50], Loss: 1.0668, Test Acc: 0.3333
Epoch [40/50], Loss: 0.9922, Test Acc: 0.3333
Epoch [50/50], Loss: 0.9300, Test Acc: 0.6667
